In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df.prices_amountmin.values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [15]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [16]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df.features.map(parse_features)

In [28]:
keys = set()


df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [32]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [43]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df.feat_brand.factorize()[0]
df['feat_color_cat'] = df.feat_color.factorize()[0]
df['feat_gender_cat'] = df.feat_gender.factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df.feat_material.factorize()[0]

df['feat_sport_cat'] = df.feat_sport.factorize()[0]
df['feat_style_cat'] = df.feat_style.factorize()[0]


for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [55]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

df[ df.brand != df.feat_brand].shape
df[ df.brand == df.feat_brand].shape
# df[ df.brand != df.feat_brand][['brand', 'feat_brand']].head()

(8846, 531)

In [57]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [86]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

feats = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_occasion_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
# feats += feats_cat
# feats = list(set(feats))

run_model(feats, model)

(-57.312178695093166, 4.273085711467046)

In [87]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2579 ± 0.0104,brand_cat
0.1006 ± 0.0064,feat_material_cat
0.0246 ± 0.0029,feat_gender_cat
0.0180 ± 0.0013,feat_brand_cat
0.0130 ± 0.0009,feat_shape_cat
0.0091 ± 0.0010,feat_metal type_cat
0.0030 ± 0.0004,feat_occasion_cat
0.0026 ± 0.0005,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [88]:
df['brand'].value_counts(normalize=True)
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'running, cross training', 'condition': 'new with box'},
       {'sport': 'soccer', 'type': 'cleats', 'color': 'yellow', 'name': 'mercurial vapor ix', 'condition': 'new without box', 'gender': 'men', 'main color': 'yellow'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'wolf grey/ivory/mystic green', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '644843032', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'wolf grey/ivory/mystic green', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '644843032', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [0]:
!git config --global user.email "tkowalski@iseno.net"
!git config --global user.name "Tomasz Kowalski"



In [0]:
!git add matrix_one/day5.ipynb
!git commit -m "day5"
!git push